In [1]:
# You should be able to just run the whole script to align
# Just run all cells
#this
import os
import yaml
import time
import numpy as np
import pandas as pd
import shelve
import sys
import random
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from scipy.optimize import minimize
sys.path.append('../../')
from geecs_python_api.controls.interface import GeecsDatabase
from geecs_python_api.controls.devices.geecs_device import GeecsDevice
# from geecs_python_api.controls.experiment.experiment import Experiment



path config ..\..\..\..\..\..\user data\Configurations.INI
database name  loasis


In [2]:
# This cell for control room setup
# DO NOT RUN

GeecsDevice.exp_info = GeecsDatabase.collect_exp_info("ControlRoom")

# set up the python objects for the geecs devices to be used in optimization
camera1 = GeecsDevice('CAM-CR-Beampointing1')
camera1.subscribe_var_values(['Target.X', 'Target.Y', 'Target2.X', 'Target2.Y'])
time.sleep(0.1)
print(camera1.state)
exposure = camera1.get('exposure')
print(f"Camera exposure is {exposure}")

target1x = camera1.get('Target.X')
target1y = camera1.get('Target.Y')
target2x = camera1.get('Target2.X')
target2y = camera1.get('Target2.Y')
print(f"Target 1 coordinates: ({int(target1x)}, {int(target1y)})")
print(f"Target 2 coordinates: ({int(target2x)}, {int(target2y)})")

centroidx = camera1.get('Centroidx')
print(centroidx)
print(camera1.state)

camera2 = GeecsDevice('CAM-CR-Beampointing2')
camera2.subscribe_var_values(['Target.X', 'Target.Y', 'Target2.X', 'Target2.Y'])
time.sleep(0.1)
print(camera2.state)
exposure = camera2.get('exposure')
print(f"Camera exposure is {exposure}")

target1x = camera2.get('Target.X')
target1y = camera2.get('Target.Y')
target2x = camera2.get('Target2.X')
target2y = camera2.get('Target2.Y')
print(f"Target 1 coordinates: ({int(target1x)}, {int(target1y)})")
print(f"Target 2 coordinates: ({int(target2x)}, {int(target2y)})")

centroidx = camera2.get('Centroidx')
print(centroidx)
print(camera2.state)

# picos = GeecsDevice('MCD-CR-Beampointing-PICOS')
# picos.subscribe_var_values(['Position.Axis 1', 'Position.Axis 2', 'Position.Axis 3', 'Position.Axis 4'])
# """For picos: beam path looks like this:
#     |      |
#     |      |
#     |      |
#     |      |
#    \|______|/
#     \      /
#    1,2     3,4 

# 1, 3 are x-dir
# 2, 4 are y-dir

# increasing pico position means increasing x, decreasing y
# """

# print("Initial pico motor positions:")
# print(f"1: {picos.get('Position.Axis 1')}")
# print(f"2: {picos.get('Position.Axis 2')}")
# print(f"3: {picos.get('Position.Axis 3')}")
# print(f"4: {picos.get('Position.Axis 4')}")
# print(picos.state)

conex = GeecsDevice('STAGE-PL1-HolyMirror')
conex.subscribe_var_values(['position'])
print("Initial conex position")
print(f"1: {conex.get('position')}")

conex2 = GeecsDevice('STAGE-PL1-HolyMirror2')
conex2.subscribe_var_values(['position'])
print("Initial conex position")
print(f"2: {conex2.get('position')}")

#this

# Negative values mean increasing mcd position decreases camera pixel value
# Camera 1 for CR
steps_per_pixel_cr1 = 26.31
steps_per_pixel_cr2 = -29.41
steps_per_pixel_cr3 = 12.04
steps_per_pixel_cr4 = -16.12

# Camera 2 for Control room (nearer camera)
steps_per_pixel_cr1 = -30.30
steps_per_pixel_cr2 = -35.71
steps_per_pixel_cr3 = -17.54
steps_per_pixel_cr4 = -22.72

steps_per_pixel_3 = 6.172
steps_per_pixel_4 = 8.70
steps_per_pixel_hex_x = 7.19e-4
steps_per_pixel_hex_y = -6.29e-4
      
# Calibrations are for cam-cr-beampointing2, which is the downstream camera
# steps_per_pixel_conex1 = 0.00014694937 # 1 inch mount
# Determined by linear regression
steps_per_pixel_conex1_camera2 = -0.0008472773555695383
steps_per_pixel_conex2_camera2 = -0.000626195225355997 # 4 inch mount

# Now for beampointing1, we need two different calibrations due to 2 different cameras
# Determined by taking ratios of movement and camera2 calibrations
steps_per_pixel_conex1_camera1 = -0.0004867208049513916
steps_per_pixel_conex2_camera1 = -0.0003148446242253497

# CR[1-4] are for testing
mcd_dict = {
#     'CR1': {
#         'device': picos, 
#         'name': 'Position.Axis 1', 
#         'alias': 'Position.Axis 1', 
#         'calib': steps_per_pixel_cr1,
#         'polyfit': None, 
#         'polyfitfine': None, 
#         'axis': 'x',
#         'min': -100000, # Software limits for beampointing
#         'max': 100000,
#        },
#     'CR2': {
#         'device': picos, 
#         'name': 'Position.Axis 2', 
#         'alias': 'Position.Axis 2', 
#         'calib': steps_per_pixel_cr2, 
#         'polyfit': None, 
#         'polyfitfine': None, 
#         'axis': 'y',
#         'min': -100000, # Software limits for beampointing
#         'max': 100000,
#         },
#     'CR3': {
#         'device': picos, 
#         'name': 'Position.Axis 3', 
#         'alias': 'Position.Axis 3', 
#         'calib': steps_per_pixel_cr3,
#         'polyfit': None, 
#         'polyfitfine': None, 
#         'axis': 'x',
#         'min': -100000, # Software limits for beampointing
#         'max': 100000,
#        },
#     'CR4': {
#         'device': picos, 
#         'name': 'Position.Axis 4', 
#         'alias': 'Position.Axis 4', 
#         'calib': steps_per_pixel_cr4,
#         'polyfit': None, 
#         'polyfitfine': None, 
#         'axis': 'y',
#         'min': -100000, # Software limits for beampointing
#         'max': 100000,
#         },
    'conex1': {
        'device': conex, 
        'name': 'position', 
        'alias': 'position', 
        'calib_cam1': steps_per_pixel_conex1_camera1,
        'calib_cam2': steps_per_pixel_conex1_camera2,
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'x',
        'min': 0, # Software limits for beampointing
        'max': 5,
    },
    'conex2': {
        'device': conex2, 
        'name': 'position', 
        'alias': 'position', 
        'calib_cam1': steps_per_pixel_conex2_camera1,
        'calib_cam2': steps_per_pixel_conex2_camera2,
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'x',
        'min': 0, # Software limits for beampointing
        'max': 5,
    }
}

SHOTS_TO_AVG = 1
TIME_BETWEEN_SHOTS = 0.1

CONEX_VEL = 0.5
CONEX_ACCEL = 4
CONEX_HYSTERESIS_SIZE = 0.03

{'fresh': True, 'shot number': None, 'GEECS device error': False}
Camera exposure is 0.00098
Target 1 coordinates: (200, 309)
Target 2 coordinates: (542, 319)
671.924683
{'fresh': True, 'shot number': 0, 'GEECS device error': False, 'Device Status': 'Initialized', '': 671.924683, 'device error': '', 'device preset': '', 'Target.X': 200.0, 'Target.Y': 309.0, 'Target2.X': 542.0, 'Target2.Y': 319.0, 'exposure': 0.00098}


KeyboardInterrupt: 

In [2]:
#this

# This cell for BELLA LPMode setup
GeecsDevice.exp_info = GeecsDatabase.collect_exp_info("Bella")
camera = GeecsDevice('CAM-PL1-LPMode')
camera.subscribe_var_values(['Target.X', 'Target.Y', 'Target2.X', 'Target2.Y'])
exposure = camera.get('exposure')
print(f"Camera exposure is {exposure}")
print(camera.state)

# picos = GeecsDevice('MCD-PL1-picosOFI2')
# picos.subscribe_var_values(['holy mirror horizontal', 'holy mirror vertical'])
# print("Initial pico motor positions:")
# print(f"3: {picos.get('Position.Axis 3')}") # holy mirror horizontal
# print(f"4: {picos.get('Position.Axis 4')}") # holy mirror vertical
# print(picos.state)

conex_x = GeecsDevice('MCD-PL1-HolyMirror1')
conex_x.subscribe_var_values(['position']) # vertical conex
print("Initial hexapod positions:")
print(f"ypos: {conex_x.get('position')}") # conex vertical
print(conex_x.state)

conex_y = GeecsDevice('MCD-PL1-HolyMirror2')
conex_y.subscribe_var_values(['position']) # horizontal conex
print("Initial hexapod positions:")
print(f"ypos: {conex_y.get('position')}") # conex vertical
print(conex_y.state)

hexapod = GeecsDevice('HEX-PL1-2')
hexapod.subscribe_var_values(['ypos', 'zpos']) # ypos for x, zpos for y
print("Initial hexapod positions:")
print(f"ypos: {hexapod.get('ypos')}") # axicon horizontal
print(f"zpos: {hexapod.get('zpos')}") # axicon vertical
print(hexapod.state)

stage = GeecsDevice('STAGE-PL1-LPmodeLong')
stage.subscribe_var_values(['position'])
print("Initial stage positions:")
print(f"ypos: {stage.get('position')}") # stage pos
print(stage.state)

Camera exposure is 5e-05
{'fresh': True, 'shot number': None, 'GEECS device error': False, 'exposure': 5e-05}
Initial hexapod positions:
ypos: 5.385233
{'fresh': True, 'shot number': None, 'GEECS device error': False, 'position': 5.385233}
Initial hexapod positions:
ypos: 6.094511
{'fresh': True, 'shot number': None, 'GEECS device error': False, 'position': 6.094511}
Initial hexapod positions:
ypos: 0.0
zpos: -1.0
{'fresh': True, 'shot number': 0, 'GEECS device error': False, 'Device Status': 'Initialized', '': '', 'device error': '', 'ypos': 0.0, 'zpos': -1.0}
Initial stage positions:
ypos: 605.000041
{'fresh': True, 'shot number': None, 'GEECS device error': False, 'position': 605.000041}


In [3]:
#this

# steps_per_pixel_3 = 6.172
# steps_per_pixel_4 = 8.70
steps_per_pixel_conex_x = -0.0001201923
steps_per_pixel_conex_y = 0.00016286644
steps_per_pixel_hex_x = 6.5e-4
steps_per_pixel_hex_y = -6.5e-4

mcd_dict = {
#     3: { # Holy Mirror pico
#         'device': picos, 
#         'name': 'Position.Axis 3', 
#         'alias': 'holy mirror horizontal', 
#         'calib': steps_per_pixel_3, 
#         'polyfit': None, 
#         'polyfitfine': None, 
#         'axis': 'x',
#         'min': picos.get('Position.Axis 3') - 5000, # Software limits for beampointing
#         'max': picos.get('Position.Axis 3') + 5000, # You can change the +- to be larger if the beam doesn't fit
#        },
#     4: { # Holy mirror pico
#         'device': picos, 
#         'name': 'Position.Axis 4', 
#         'alias': 'holy mirror vertical', 
#         'calib': steps_per_pixel_4, 
#         'polyfit': None, 
#         'polyfitfine': None, 
#         'axis': 'y', 
#         'min': picos.get('Position.Axis 4') - 5000,
#         'max': picos.get('Position.Axis 4') + 5000,
#        },
    'HM_x': { # Holy mirror x
        'device': conex_x, 
        'name': 'position', 
        'alias': 'position', 
        'calib': steps_per_pixel_conex_x, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'x',
        'min': 0,
        'max': 10,
    },
    'HM_y': { # Holy mirror y
        'device': conex_y, 
        'name': 'position', 
        'alias': 'position', 
        'calib': steps_per_pixel_conex_y, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'x',
        'min': 0,
        'max': 10,
    },
    'axicon_x': {
        'device': hexapod, 
        'name': 'ypos', 
        'alias': 'ypos', 
        'calib': steps_per_pixel_hex_x, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'x',
        'min': hexapod.get('ypos') - 0.4,
        'max': hexapod.get('ypos') + 0.4,
    },
    'axicon_y': {
        'device': hexapod, 
        'name': 'zpos', 
        'alias': 'zpos', 
        'calib': steps_per_pixel_hex_y, 
        'polyfit': None, 
        'polyfitfine': None, 
        'axis': 'y',
        'min': hexapod.get('zpos') - 0.4,
        'max': hexapod.get('zpos') + 0.4,
    },
}

In [4]:
#this
# Hyperparameters
SHOTS_TO_AVG = 3
TIME_BETWEEN_SHOTS = 1 # seconds

In [5]:
def set_with_limit(mcd, value, min_val=None, max_val=None):
    """
    Set a parameter with software limits, default is no limit
    This is a blocking function since we are setting
    
    mcd: device dictionary in mcd_dict
    value: value to set to
    min_val, max_val: you can temporarily override the software limits by specifying
    a non-null value
    
    return: actual value set
    """
    if not min_val:
        min_val = mcd['min']
    if not max_val:
        max_val = mcd['max']
        
    if min_val > max_val:
        raise Exception('min_val must be smaller than max_val')
        
    mcd['device'].set(mcd['name'], max(min_val, min(max_val, value)))
    return max(min_val, min(max_val, value))

In [6]:
# Hysteresis fix for conex
def conex_move_time(move_size, vel, accel):
    """
    Estimates the time for a conex move
    """
    t_accel = vel/accel
    t_vel = (move_size / 2 - 0.5 * accel * t_accel**2) / vel
    return (t_vel + t_accel) * 2 + 1.5 # Arbitrary extra time for move = 2

def conex_move_hysteresis(conex_name, new_pos, eps):
    """
    Moves a conex motor with hysteresis compensation in the positive direction
    conex_name: name of the conex device in the mcd_dict
    new_pos: position to move to
    eps: size of hysteresis compensation
    """
    conex = mcd_dict[conex_name]
    cur_pos = conex['device'].get(conex['name'])
    print(f"new pos: {new_pos}, cur_pos: {cur_pos}")
    set_with_limit(conex, new_pos)
    if new_pos > cur_pos: # moving in + direction
        print("forward", conex_move_time(new_pos + eps - cur_pos, CONEX_VEL, CONEX_ACCEL))
        time.sleep(conex_move_time(new_pos + eps - cur_pos, CONEX_VEL, CONEX_ACCEL))
        time.sleep(0.5) # wait a bit for the motor to turn around
        print("backwards", conex_move_time(eps, CONEX_VEL, CONEX_ACCEL))
        time.sleep(conex_move_time(eps, CONEX_VEL, CONEX_ACCEL))
    else: # moving in - direction
        time.sleep(conex_move_time(cur_pos - new_pos, CONEX_VEL, CONEX_ACCEL))

In [7]:
#this
# Beam pointing onto target
def obj_func(targetx, targety, beamx, beamy):
    return (targetx - beamx)**2 + (targety - beamy)**2

def move_random(pico_num, picos, dmin, dmax):
    """
    pico_num: an index for mcd_dict
    picos: GEECS picos object
    moves the pico between [dmin, dmax] randomly from its original position
    """
    picomotor = mcd_dict[pico_num]
    new_pos = picos.state[picomotor['name']] + random.randint(dmin, dmax)
    print(f"moving motor {pico_num} to position {new_pos}")
    picos.set(picomotor['name'], int(picos.state[picomotor['name']] + random.randint(dmin, dmax)))
    return

def get_centroid(camera, num_shots, time_between_shots):
    """
    Returns (x,y) of centroid of beam
    """
    centroidx = []
    centroidy = []
    # Average over SHOTS_TO_AVG shots
    for i in range(num_shots):
        # While loops since sometimes the camera doesn't send back a value for centroid
        centroidx1, centroidy1 = None, None
        while not isinstance(centroidx1, float):
            centroidx1 = camera.get('Centroidx')
#             print(centroidx1)
        centroidx.append(centroidx1)
        while not isinstance(centroidy1, float):
            centroidy1 = camera.get('Centroidy')
        centroidy.append(centroidy1)
        print(f"Centroid is {centroidx1}, {centroidy1}")
        time.sleep(time_between_shots)

    centroidx = np.mean(np.array(centroidx))
    centroidy = np.mean(np.array(centroidy))
    return centroidx, centroidy

def move_onto_target(camera, obj_target, opt_rate, pico1, pico2, target, fit='linear', 
                     fine_thresh=10, hexapod_flag=True, verbose=True):
    """
    Moves a beam onto a target with pico1 and pico2
    
    camera: GEECS camera object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    pico1, pico2: choose 2 picos, they must be one in the x direction and one
    in the y direction, these should be keys in the mcd_dict
    target: an integer 1 or 2, this is which target we are pointing at
    fit: if linear, use linear calib (steps_per_pixel)
    if poly, use polynomial fit (these should be PolynomialModel instances inside mcd_dict)
    hexapod_flag: True if aligning hexapods, False if aligning picos
    (This is because hexapods take floats as inputs, and picos take ints as inputs)
    
    returns: iterations, obj_func()
    """
    if hexapod_flag:
        print("Moving a hexapod")
    else:
        print("Moving pico motors")
    
    # Helper function to calculate objective function
    def obj():
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
            
        if target == 1:
            return obj_func(camera.state['Target.X'], camera.state['Target.Y'], 
                            centroidx, centroidy)
        elif target == 2:
            return obj_func(camera.state['Target2.X'], camera.state['Target2.Y'], 
                            centroidx, centroidy)
        else:
            raise Exception('target must be either 1 or 2')
    
    # Helper function to get target position
    def target_pos():
        if target == 1:
            return camera.state['Target.X'], camera.state['Target.Y']
        elif target == 2:
            return camera.state['Target2.X'], camera.state['Target2.Y']
        else:
            raise Exception('target must be either 1 or 2')
            
    # giving mcds more convenient names
    if mcd_dict[pico1]['axis'] == 'x' and mcd_dict[pico2]['axis'] == 'y':
        mcd_x = mcd_dict[pico1] # picomotor means the motor dictionary entry
        mcd_y = mcd_dict[pico2]
    elif mcd_dict[pico1]['axis'] == 'y' and mcd_dict[pico2]['axis'] == 'x':
        mcd_x = mcd_dict[pico2]
        mcd_y = mcd_dict[pico1]
    else:
        raise Exception('PICOs must span both x and y axes')
        
    # Move mcds until beam is close enough to target
    iterations = 0
    target_x, target_y = target_pos()
    while obj() > obj_target:
        iterations += 1
        if verbose:
            print(f"Iteration {iterations}")
        pico_x = mcd_x['device'].state[mcd_x['alias']]
        pico_y = mcd_y['device'].state[mcd_y['alias']]
        if verbose:
            print(f"Original pico position is ({int(pico_x)}, {int(pico_y)})")
        centroidx = None
        centroidy = None
        # While loops since sometimes the camera doesn't send back a value for centroid
        while not isinstance(centroidx, float):
            centroidx = camera.get('Centroidx')
        while not isinstance(centroidy, float):
            centroidy = camera.get('Centroidy')
        dx = target_x - centroidx
        dy = target_y - centroidy
        if verbose:
            print(f"original centroid position = ({centroidx}, {centroidy})")
            print(f"dx = {dx}, dy = {dy}")
        if fit == "linear":
            move_x = (dx) * mcd_x['calib'] * opt_rate
            move_y = (dy) * mcd_y['calib'] * opt_rate
        if fit == "poly":
            if np.abs(dx) > fine_thresh:
                move_x = mcd_x['polyfit'].predict_motor_move(dx) * opt_rate
            else:
                move_x = mcd_x['polyfitfine'].predict_motor_move(dx) * opt_rate
            if np.abs(dy) > fine_thresh:
                move_y = mcd_y['polyfit'].predict_motor_move(dy) * opt_rate
            else:
                move_y = mcd_y['polyfitfine'].predict_motor_move(dy) * opt_rate
        if verbose:
            if hexapod_flag:
                print(f"Moving by ({move_x}, {move_y})")
            else:
                print(f"Moving by ({int(move_x)}, {int(move_y)})")
            print(f"Moving to ({pico_x + move_x}, {pico_y + move_y})")
        if hexapod_flag: # hexapods take float moves
            set_with_limit(mcd_x, np.round(pico_x + move_x, 5))
            set_with_limit(mcd_y, np.round(pico_y + move_y, 5))
            if verbose:
                print(f"Final position is ({mcd_x['device'].state[mcd_x['alias']]}, "
                      f"{mcd_x['device'].state[mcd_y['alias']]})")
        else: # picos take int moves
            set_with_limit(mcd_x, int(pico_x + move_x))
            set_with_limit(mcd_y, int(pico_y + move_y))
            if verbose:
                print(f"Final position is ({int(mcd_x['device'].state[mcd_x['alias']])}, "
                      f"{int(mcd_x['device'].state[mcd_y['alias']])})")
        time.sleep(0.1)
        
    
    return iterations, obj()

def move_onto_target_x(camera, obj_target, opt_rate, mcd1, target):
    """
    Moves a beam onto a target with pico1 and pico2
    
    camera: GEECS camera object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    pico1, pico2: choose 2 picos, they must be one in the x direction and one
    in the y direction, these should be keys in the mcd_dict
    target: an integer 1 or 2, this is which target we are pointing at
    fit: if linear, use linear calib (steps_per_pixel)
    if poly, use polynomial fit (these should be PolynomialModel instances inside mcd_dict)
    hexapod_flag: True if aligning hexapods, False if aligning picos
    (This is because hexapods take floats as inputs, and picos take ints as inputs)
    
    returns: iterations, obj_func()
    """
    
    # Helper function to calculate objective function
    def obj():
        centroidx, centroidy = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
            
        if target == 1:
            return (camera.state['Target.X'] - centroidx)**2
        elif target == 2:
            return (camera.state['Target2.X'] - centroidx)**2
        else:
            raise Exception('target must be either 1 or 2')
    
    # Helper function to get target position
    def target_pos():
        if target == 1:
            return camera.state['Target.X'], camera.state['Target.Y']
        elif target == 2:
            return camera.state['Target2.X'], camera.state['Target2.Y']
        else:
            raise Exception('target must be either 1 or 2')
    
    mcd_x = mcd_dict[mcd1]
        
    # Move mcds until beam is close enough to target
    iterations = 0
    target_x, target_y = target_pos()
    while obj() > obj_target:
        iterations += 1
        print(f"Iteration {iterations}")
        pico_x = mcd_x['device'].state[mcd_x['alias']]
        print(f"Original pico position is ({float(pico_x)})")
        centroidx = None
        # While loops since sometimes the camera doesn't send back a value for centroid
        while not isinstance(centroidx, float):
            centroidx = camera.get('Centroidx')
        dx = target_x - centroidx
        print(f"original centroid position = ({centroidx})")
        print(f"dx = {dx}")
        move_x = (dx) * mcd_x['calib'] * opt_rate
        conex_move_hysteresis(mcd1, np.round(pico_x + move_x, 5), CONEX_HYSTERESIS_SIZE)
        
        print(f"Final position is ({mcd_x['device'].state[mcd_x['alias']]})")
        time.sleep(0.1)
        
    
    return iterations, obj()
    
    

In [16]:
#this
def align_2_targets(mcd1, mcd2, mcd1hex, mcd2hex, camera, obj_target, 
                    opt_rate, fit='linear', fit_thresh=10):
    """
    Align to the further upstream position first (larger stage position)
    mcd1 should be picos and mcd2 should be hexapod in 1BL and 2BL
    
    Stage positions are extracted from crosshair label names
    
    This function assumes higher stage pos = further upstream
    
    mcd1: list like object of 2 labels to control motion control device 1
    This mcd should correspond to crosshair 1
    mcd2: list like object of 2 labels to control motion control device 2
    This mcd should correspond to crosshair 2
    mcd1hex, mcd2hex: True if they are hexapods, False if they aren't
    camera: GEECS camera object
    picos: GEECS picos object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    fit: if linear, use linear calib (steps_per_pixel)
    if poly, use polynomial fit (these should be PolynomialModel instances inside picos_dict)
    fit_thresh: threshold between fine and rough poly fits
    """
    stage_pos1_name = camera.get('Crosshair.Label1')
    stage_pos2_name = camera.get('Crosshair.Label2')
    # Extracting position 
    stage_pos1 = float(''.join([i for i in stage_pos1_name.split() if (i.isdigit() or i == '.')]))
    stage_pos2 = float(''.join([i for i in stage_pos2_name.split() if (i.isdigit() or i == '.')]))
    
    stage_upstream = max(stage_pos1, stage_pos2)
    stage_downstream = min(stage_pos1, stage_pos2)
    
    upstream_target = 1 if stage_pos1 > stage_pos2 else 2
    downstream_target = 1 if stage_pos1 < stage_pos2 else 2
    
    mcd_upstream = mcd1 if stage_pos1 > stage_pos2 else mcd2
    mcd_downstream = mcd1 if stage_pos1 < stage_pos2 else mcd2
    
    hex_flag_upstream = mcd1hex if stage_pos1 > stage_pos2 else mcd2hex
    hex_flag_downstream = mcd1hex if stage_pos1 < stage_pos2 else mcd2hex
    
    upstream_aligned, downstream_aligned = False, False
    while not (upstream_aligned and downstream_aligned):
        # Align to upstream first
        stage.set('position', stage_upstream)
        print(f"Stage position moved to {stage_upstream}")
        print("Moving mcd1")
        iterations, _ = move_onto_target(camera, obj_target, opt_rate, *mcd_upstream, upstream_target, 
                                         fit='linear', hexapod_flag=hex_flag_upstream)
        upstream_aligned = True
        if iterations > 0:
            downstream_aligned = False
        print('\n\n')
        
        stage.set('position', stage_downstream)
        print(f"Stage position moved to {stage_downstream}")
        print("Moving mcd2")
        iterations, _ = move_onto_target(camera, obj_target, opt_rate, *mcd_downstream, downstream_target, 
                                         fit='linear', hexapod_flag=hex_flag_downstream)
        downstream_aligned = True
        if iterations > 0:
            upstream_aligned = False
            
        print("\n\n")
        
    return
    

In [17]:
def align_2_targets_at_once_cr(mcd1, mcd2, mcd1hex, mcd2hex, camera1, camera2, obj_target, 
                               opt_rate, stage_pos1, stage_pos2, 
                               calib1_length, calib2_length, fit='linear', fit_thresh=10):
    """
    Uses 2 different calibrations to align to 2 targets at once
    
    mcd1: list like object of 2 labels to control motion control device 1
    mcd2: list like object of 2 labels to control motion control device 2
    mcd1hex, mcd2hex: True if they are hexapods, False if they aren't
    camera: GEECS camera object
    picos: GEECS picos object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    stage_pos1: should correspond to target1
    stage_pos2: should correspond to target2
    STAGE_POS1 SHOULD BE FURTHER DOWNSTREAM THAN STAGE_POS2
    calib_1_length: path length from mcd1 to stage_pos1
    calib_2_length: path length from mcd2 to stage_pos1
    fit: if linear, use linear calib (steps_per_pixel)
    if poly, use polynomial fit (these should be PolynomialModel instances inside picos_dict)
    fit_thresh: threshold between fine and rough poly fits
    """
    # x1 = a1 * hex_x + b1 * pico_x
    # x2 = a2 * hex_x + b2 * pico_x
    # Solve for hex x and pico x
#     while True:
    centroid1_x, centroid1_y = get_centroid(camera1, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    centroid2_x, centroid2_y = get_centroid(camera2, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)

    for mcd_index in mcd1:
        mcd = mcd_dict[mcd_index]
        if mcd['axis'] == 'x':
            calib1_pos1_x = mcd['calib']
            mcd1_x = mcd
        if mcd['axis'] == 'y':
            calib1_pos1_y = mcd['calib']
            mcd1_y = mcd
    for mcd_index in mcd2:
        mcd = mcd_dict[mcd_index]
        if mcd['axis'] == 'x':
            calib2_pos1_x = mcd['calib']
            mcd2_x = mcd
        if mcd['axis'] == 'y':
            calib2_pos1_y = mcd['calib']
            mcd2_y = mcd

    calib1_pos2_x = -calib1_pos1_x * (calib1_length + np.abs(stage_pos1 - stage_pos2)) / calib1_length
    calib1_pos2_y = calib1_pos1_y * (calib1_length + np.abs(stage_pos1 - stage_pos2)) / calib1_length
    calib2_pos2_x = -calib2_pos1_x * (calib2_length + np.abs(stage_pos1 - stage_pos2)) / calib2_length
    calib2_pos2_y = calib2_pos1_y * (calib2_length + np.abs(stage_pos1 - stage_pos2)) / calib2_length

    # X matrix equation (A * \vec{pos} = \vec{b})
    # A is matrix of calibrations
    # pos = positions/values of mcds
    # b = pixel difference in target vs centroid
    # Must do this for x and y direction

    # x-dir
    A = 1 / np.array([
        [calib1_pos1_x, calib2_pos1_x],
        [calib1_pos2_x, calib2_pos2_x]
    ])
    b = np.array([
        [centroid1_x - camera1.state['Target.X']], [centroid2_x - camera2.state['Target2.X']]
    ]) # take reciprocal because calib numbers in steps/pixel instead of pixels/step
    print("Solving equation Ax=b")
    print("A =", A)
    print("b =", b)
    pos = np.matmul(np.linalg.inv(A), b)
    mcd1_move_x = pos[0][0] * opt_rate
    mcd2_move_x = pos[1][0] * opt_rate
    
    print(f"mcd1_move_x: {mcd1_move_x}")
    print(f"mcd2_move_x: {mcd2_move_x}")
    mcd1_x_new_pos = mcd1_move_x + mcd1_x['device'].state[mcd1_x['alias']]
    if not mcd1hex:
        mcd1_x_new_pos = int(mcd1_x_new_pos)
    mcd2_x_new_pos = mcd2_move_x + mcd2_x['device'].state[mcd2_x['alias']]
    if not mcd2hex:
        mcd2_x_new_pos = int(mcd2_x_new_pos)

    # y-dir
    A = 1 / np.array([
        [calib1_pos1_y, calib2_pos1_y],
        [calib1_pos2_y, calib2_pos2_y]
    ]) # take reciprocal because calib numbers in steps/pixel instead of pixels/step
    b = np.array([
        [centroid1_y - camera1.state['Target.Y']], [centroid2_y - camera2.state['Target2.Y']]
    ])
    print("Solving equation Ax=b")
    print("A =", A)
    print("b =", b)
    pos = np.matmul(np.linalg.inv(A), b)
    mcd1_move_y = pos[0][0] * opt_rate
    mcd2_move_y = pos[1][0] * opt_rate
    
    print(f"mcd1_move_y: {mcd1_move_y}")
    print(f"mcd2_move_y: {mcd2_move_y}")
    mcd1_y_new_pos = mcd1_move_y + mcd1_y['device'].state[mcd1_y['alias']]
    if not mcd1hex:
        mcd1_y_new_pos = int(mcd1_y_new_pos)
    mcd2_y_new_pos = mcd2_move_y + mcd2_y['device'].state[mcd2_y['alias']]
    if not mcd2hex:
        mcd2_y_new_pos = int(mcd2_y_new_pos)

    # Move all 4 motors at once
#     mcd1_x['device'].set(mcd1_x['name'], mcd1_x_new_pos)
#     mcd2_x['device'].set(mcd2_x['name'], mcd2_x_new_pos)
#     mcd1_y['device'].set(mcd1_y['name'], mcd1_y_new_pos)
#     mcd2_y['device'].set(mcd2_y['name'], mcd2_y_new_pos)

    # Check to see if moved to correct positions
    centroid1_x, centroid1_y = get_centroid(camera1, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    centroid2_x, centroid2_y = get_centroid(camera2, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)

    obj1 = obj_func(camera1.state['Target.X'], camera1.state['Target.Y'], centroid1_x, centroid1_y)
    obj2 = obj_func(camera2.state['Target2.X'], camera2.state['Target2.Y'], centroid2_x, centroid2_y)

    print(f"obj1: {obj1}")
    print(f"obj2: {obj2}")
#     if (obj1 < obj_target) and (obj2 < obj_target):
#         break
    
    return
    
    
    

In [18]:
def align_2_targets_at_once_cr_y_close(mcd1, camera1, camera2, eps=1e-3):
    """
    Aligns both targets by moving one of the mcds by 1 step at a time
    Both beams must be either above or below the target
    """
    centroid1_x, centroid1_y = get_centroid(camera1, 1, 0)
    centroid1_x, centroid2_y = get_centroid(camera2, 1, 0)
    cam1_diff = camera1.state['Target.Y'] - centroid1_y
    cam2_diff = camera2.state['Target.Y'] - centroid2_y
    
    step = 0
    if cam2_diff > 0 and cam1_diff > 0:
        sign = -1
        step = eps * -1
    elif cam2_diff < 0 and cam1_diff < 0:
        sign = 1
        step = eps
    
    positions = [mcd_dict[mcd1]['device'].state[mcd_dict[mcd1]['alias']]]
    objs = [cam1_diff**2 + cam2_diff**2]
    if step != 0:
        # precondition is satisfied
        while sign * (camera1.state['Target.Y'] - centroid1_y) < 0 or sign * (camera2.state['Target.Y'] - centroid2_y) < 0:
            conex_move_hysteresis(mcd1, mcd_dict[mcd1]['device'].state[mcd_dict[mcd1]['alias']] + step, CONEX_HYSTERESIS_SIZE)
            time.sleep(1)
            centroid1_x, centroid1_y = get_centroid(camera1, 1, 0)
            centroid1_x, centroid2_y = get_centroid(camera2, 1, 0)
            cam1_diff = camera1.state['Target.Y'] - centroid1_y
            cam2_diff = camera2.state['Target.Y'] - centroid2_y
            positions.append(positions[-1] + step)
            objs.append(cam1_diff**2 + cam2_diff**2)
    
        # move to optimal according to previous moves
        objs = np.array(objs)
        opt_position = positions[np.argmin(objs)]
        conex_move_hysteresis(mcd1, opt_position, CONEX_HYSTERESIS_SIZE)
        print(positions)
        print(objs)
        return
    else:
        print("Both targets must be above or below the target")
        return

In [19]:
def align_2_targets_at_once_cr_y(mcd1, mcd2, camera1, camera2, rough_target, 
                               opt_rate_thresh, fit='linear', fit_thresh=10):
    """
    Uses 2 different calibrations to align to 2 targets at once
    
    mcd1: list like object of 2 labels to control motion control device 1
    mcd2: list like object of 2 labels to control motion control device 2
    mcd1hex, mcd2hex: True if they are hexapods, False if they aren't
    camera: GEECS camera object
    picos: GEECS picos object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate_thresh: threshold for significantly smaller optimization, using a sigmoid function
    stage_pos1: should correspond to target1
    stage_pos2: should correspond to target2
    STAGE_POS1 SHOULD BE FURTHER DOWNSTREAM THAN STAGE_POS2
    calib_1_length: path length from mcd1 to stage_pos1
    calib_2_length: path length from mcd2 to stage_pos1
    fit: if linear, use linear calib (steps_per_pixel)
    if poly, use polynomial fit (these should be PolynomialModel instances inside picos_dict)
    fit_thresh: threshold between fine and rough poly fits
    """
    # x1 = a1 * hex_x + b1 * pico_x
    # x2 = a2 * hex_x + b2 * pico_x
    # Solve for hex x and pico x
    while True:
        centroid1_x, centroid1_y = get_centroid(camera1, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        centroid1_x, centroid2_y = get_centroid(camera2, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        
        obj1 = (camera1.state['Target.Y'] - centroid1_y)**2
        obj2 = (camera2.state['Target.Y'] - centroid2_y)**2

        print(f"obj1: {obj1}")
        print(f"obj2: {obj2}")
        if (obj1 < rough_target) and (obj2 < rough_target):
            if (camera1.state['Target.Y'] - centroid1_y) * (camera2.state['Target.Y'] - centroid2_y) > 0:
                break

        calib_mcd1_cam1_y = mcd_dict[mcd1]['calib_cam1']
        mcd1_y = mcd_dict[mcd1]

        calib_mcd2_cam1_y = mcd_dict[mcd2]['calib_cam1']
        mcd2_y = mcd_dict[mcd2]

        calib_mcd1_cam2_y = mcd_dict[mcd1]['calib_cam2']
        calib_mcd2_cam2_y = mcd_dict[mcd2]['calib_cam2']

        # X matrix equation (A * \vec{pos} = \vec{b})
        # A is matrix of calibrations
        # pos = positions/values of mcds
        # b = pixel difference in target vs centroid
        # Must do this for x and y direction

        # y-dir
        A = 1 / np.array([
            [calib_mcd1_cam1_y, calib_mcd2_cam1_y],
            [calib_mcd1_cam2_y, calib_mcd2_cam2_y]
        ]) # take reciprocal because calib numbers in steps/pixel instead of pixels/step
        b = np.array([
            [camera1.state['Target.Y'] - centroid1_y], [camera2.state['Target.Y'] - centroid2_y]
        ])
        print("Solving equation Ax=b")
        print("A =", A)
        print("b =", b)
        pos = np.matmul(np.linalg.inv(A), b)
        print("pos = ", pos)
        larger_move = max(abs(pos[0][0]), abs(pos[1][0]))
        # Sigmoid optimization rate
        opt_rate = min(1, abs(larger_move/opt_rate_thresh))
        print("Opt rate:", opt_rate)

        # Make sure move is at least 0.001
        if pos[0][0] > 0:
            mcd1_move_y = max(pos[0][0] * opt_rate, 0.001)
        else:
            mcd1_move_y = min(pos[0][0] * opt_rate, -0.001)
        if pos[1][0] > 0:
            mcd2_move_y = max(pos[1][0] * opt_rate, 0.001)
        else:
            mcd2_move_y = min(pos[1][0] * opt_rate, -0.001)

        print(f"mcd1_move_y: {mcd1_move_y}")
        print(f"mcd2_move_y: {mcd2_move_y}")
        print(mcd1_y['device'].state[mcd1_y['alias']])
        mcd1_y_new_pos = mcd1_move_y + mcd1_y['device'].state[mcd1_y['alias']]
        mcd2_y_new_pos = mcd2_move_y + mcd2_y['device'].state[mcd2_y['alias']]

        print(f"mcd1 new position: {mcd1_y_new_pos}")
        print(f"mcd2 new position: {mcd2_y_new_pos}")

    #     Move all 4 motors at once
    #     mcd1_y['device'].set(mcd1_y['name'], mcd1_y_new_pos)
    #     mcd2_y['device'].set(mcd2_y['name'], mcd2_y_new_pos)
        conex_move_hysteresis(mcd1, mcd1_y_new_pos, CONEX_HYSTERESIS_SIZE)
        time.sleep(1)
        conex_move_hysteresis(mcd2, mcd2_y_new_pos, CONEX_HYSTERESIS_SIZE)

        time.sleep(1)
        # Check to see if moved to correct positions
        centroid1_x, centroid1_y = get_centroid(camera1, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        centroid2_x, centroid2_y = get_centroid(camera2, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)

        obj1 = (camera1.state['Target.Y'] - centroid1_y)**2
        obj2 = (camera2.state['Target.Y'] - centroid2_y)**2

        print(f"obj1: {obj1}")
        print(f"obj2: {obj2}")
        if (obj1 < rough_target) and (obj2 < rough_target):
            # fine alignment only possible if both x and y directions 
            if (camera1.state['Target.Y'] - centroid1_y) * (camera2.state['Target.Y'] - centroid2_y) > 0:
                break
                
    print("-"*20)
    print("Moving to fine alignment")
    print("-"*20)
    align_2_targets_at_once_cr_y_close(mcd1, camera1, camera2, eps=1e-3)
    
    centroid1_x, centroid1_y = get_centroid(camera1, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    centroid2_x, centroid2_y = get_centroid(camera2, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)

    obj1 = (camera1.state['Target.Y'] - centroid1_y)**2
    obj2 = (camera2.state['Target.Y'] - centroid2_y)**2
    
    return obj1, obj2
    
    
    

In [28]:
def align_2_targets_at_once(close_mcd_x, close_mcd_y, far_mcd_x, far_mcd_y, camera, obj_target, 
                            rough_target, opt_rate, stage_pos_close, stage_pos_far, matrix,
                            fit='linear', fit_thresh=10):
    """
    Uses 2 different calibrations to align to 2 targets at once
    When using this function, make sure that "Target.X" and "Target.Y" are the upstream camera
    position and "Target2.X"/"Target2.Y" are the downstream position
    
    close_mcd_x, close_mcd_y: dictionary entries for upstream mcd x and y controllers
    far_mcd_x, far_mcd_y: dictionary entries for downstream mcd x and y controllers
    It is assumed that the upstream mcd "aligns to" the closer position, while the downstream mcd
    "aligns to" the further position
    camera: GEECS camera object
    picos: GEECS picos object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    stage_pos_close: closer numerical value for stage position
    stage_pos_far: far numerical value for stage position
    STAGE_POS1 SHOULD BE FURTHER DOWNSTREAM THAN STAGE_POS2
    calib_close_length: path length from close mcd to stage_pos_close
    calib_far_length: path length from far mcd to stage_pos_close
    fit: if linear, use linear calib (steps_per_pixel)
    if poly, use polynomial fit (these should be PolynomialModel instances inside picos_dict)
    fit_thresh: threshold between fine and rough poly fits
    """
    # x1 = a1 * hex_x + b1 * pico_x
    # x2 = a2 * hex_x + b2 * pico_x
    # Solve for hex x and pico x
    begin_time = time.time()
    
    while True:
        t1 = time.time()
        stage.set('position', stage_pos_close)
        print(f"Time for stage to move: {time.time() - t1}")
        time.sleep(3)
        print(f"Time after sleep: {time.time() - t1}")
        centroid_close_x, centroid_close_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        t1 = time.time()
        stage.set('position', stage_pos_far)
        print(f"Time for stage to move: {time.time() - t1}")
        time.sleep(3)
        print(f"Time after sleep: {time.time() - t1}")
        centroid_far_x, centroid_far_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)

        # X matrix equation (A * \vec{pos} = \vec{b})
        # A is matrix of calibrations
        # pos = positions/values of mcds
        # b = pixel difference in target vs centroid
        # Must do this for x and y direction

        # x-dir
        A = 1 / matrix # take reciprocal because calib numbers in steps/pixel instead of pixels/step
        print("Matrix A is (X-dir):", A)
        b = np.array([
            [centroid_close_x - camera.state['Target.X']],
            [centroid_far_y - camera.state['Target2.Y']],
            [centroid_far_x - camera.state['Target2.X']],
            [centroid_far_y - camera.state['Target2.Y']],
        ])
        pos = np.matmul(np.linalg.inv(A), b)
        mcd_close_move_x = pos[0][0] * opt_rate
        mcd_far_move_x = pos[2][0] * opt_rate
        mcd_close_move_y = pos[1][0] * opt_rate
        mcd_far_move_y = pos[3][0] * opt_rate
        mcd_close_x_new_pos = mcd_close_move_x + close_mcd_x['device'].state[close_mcd_x['alias']]
        mcd_far_x_new_pos = mcd_far_move_x + far_mcd_x['device'].state[far_mcd_x['alias']]
        mcd_close_y_new_pos = mcd_close_move_y + close_mcd_y['device'].state[close_mcd_y['alias']]
        mcd_far_y_new_pos = mcd_far_move_y + far_mcd_y['device'].state[far_mcd_y['alias']]

        print("axicon x new pos:", mcd_close_x_new_pos)
        print("axicon y new pos:", mcd_close_y_new_pos)
        print("HM x new pos:", mcd_far_x_new_pos)
        print("HM y new pos:", mcd_far_y_new_pos)
        # Move all 4 motors at once
#         close_mcd_x['device'].set(close_mcd_x['name'], mcd_close_x_new_pos)
#         far_mcd_x['device'].set(far_mcd_x['name'], mcd_far_x_new_pos)
#         close_mcd_y['device'].set(close_mcd_y['name'], mcd_close_y_new_pos)
#         far_mcd_y['device'].set(far_mcd_y['name'], mcd_far_y_new_pos)
        
        # Check to see if moved to correct positions
        stage.set('position', stage_pos_close)
        centroid_close_x, centroid_close_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        stage.set('position', stage_pos_far)
        centroid_far_x, centroid_far_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        
        obj1 = obj_func(camera.state['Target.X'], camera.state['Target.Y'], centroid_close_x, centroid_close_y)
        obj2 = obj_func(camera.state['Target2.X'], camera.state['Target2.Y'], centroid_far_x, centroid_far_y)
        
        if (obj1 < rough_target) and (obj2 < rough_target):
            # fine alignment only possible if both positions are positive or negative w.r.t target
            if (camera.state['Target.Y'] - centroid_close_y) * (camera.state['Target2.Y'] - centroid_far_y) > 0:
                if (camera.state['Target.X'] - centroid_close_x) * (camera.state['Target2.X'] - centroid_far_x) > 0:
                    break
                    
    align_2_targets_at_once_close(close_mcd_x, camera, stage_pos_close, stage_pos_far, eps=1e-3, delta_steps=3, axis="X")
    align_2_targets_at_once_close(close_mcd_y, camera, stage_pos_close, stage_pos_far, eps=1e-3, delta_steps=3, axis="Y")
    
    stage.set('position', stage_pos_close)
    centroid1_x, centroid1_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    stage.set('position', stage_pos_far)
    centroid2_x, centroid2_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    print("Close delta x:", centroid1_x - camera.state['Target.X'])
    print("Close delta y:", centroid1_y - camera.state['Target.Y'])
    print("Far delta x:", centroid2_x - camera.state['Target2.X'])
    print("Far delta y:", centroid2_y - camera.state['Target2.X'])
    print(f"Alignment took {time.time() - begin_time} seconds")
    return
    
    
    

In [2]:
def move_pico_to_pixel(camera, obj_target, opt_rate, pico, target_pix, axis='x'):
    """
    Moves a beam onto a target with pico1 and pico2
    
    camera: GEECS camera object
    obj_target: stop optimizing when we are less than obj_target pixels in the given axis
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    pico: object in the mcd dict, this should be a picomotor
    target_pix: pixel to move to
    axis: axis to move the pico on
    
    returns: iterations, obj_func()
    """
    
    # Helper function to get target position
    def get_centroid_pix():
        if axis == 'x':
            return camera.get('Centroidx')
        elif axis == 'y':
            return camera.get('Centroidy')
        else:
            raise Exception('axis must be either 1 or 2')
        
    # Move mcds until beam is close enough to target
    iterations = 0
    while obj() > obj_target:
        iterations += 1
        print(f"Iteration {iterations}")
        pico_pos = pico['device'].state[pico['alias']]
        print(f"Original pico position is ({float(pico_pos)})")
        centroid = get_centroid_pix()
        # While loops since sometimes the camera doesn't send back a value for centroid
        while not isinstance(centroid, float):
            centroid = get_centroid_pix()
        dx = target_pix - centroid
        print(f"original centroid position = ({centroid})")
        print(f"dx = {dx}")
        move = (dx) * pico['calib'] * opt_rate
        set_with_limit(pico, int(pico_pos + move))
        
        print(f"Final position is ({pico['device'].state[pico['alias']]})")
        time.sleep(0.1)
        
    
    return iterations, obj()
    
    
def move_axicon_to_pixel(camera, obj_target, opt_rate, axicon, target_pix, axis='x'):
    """
    Moves a beam onto a target with pico1 and pico2
    
    camera: GEECS camera object
    obj_target: stop optimizing when we are less than obj_target pixels in the given axis
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    axicon: object in the mcd dict, this should be a picomotor
    target_pix: pixel to move to
    axis: axis to move the pico on
    
    returns: iterations, obj_func()
    """
    
    # Helper function to get target position
    def get_centroid_pix():
        if axis == 'x':
            return camera.get('Centroidx')
        elif axis == 'y':
            return camera.get('Centroidy')
        else:
            raise Exception('axis must be either 1 or 2')
        
    # Move mcds until beam is close enough to target
    iterations = 0
    while obj() > obj_target:
        iterations += 1
        print(f"Iteration {iterations}")
        axicon_pos = axicon['device'].state[axicon['alias']]
        print(f"Original pico position is ({float(axicon_pos)})")
        centroid = get_centroid_pix()
        # While loops since sometimes the camera doesn't send back a value for centroid
        while not isinstance(centroid, float):
            centroid = get_centroid_pix()
        dx = target_pix - centroid
        print(f"original centroid position = ({centroid})")
        print(f"dx = {dx}")
        move = (dx) * axicon['calib'] * opt_rate
        set_with_limit(axicon, axicon_pos + move)
        
        print(f"Final position is ({axicon['device'].state[axicon['alias']]})")
        time.sleep(0.1)
        
    
    return iterations, obj()
    

In [3]:
def align_2_targets_at_once(mcd_1x, mcd_1y, mcd_2x, mcd_2y, camera, obj_target, 
                            opt_rate, stage_pos_a, stage_pos_b, lambda_1,
                            lambda_2, fit='linear', fit_thresh=10):
    """
    Uses 2 different calibrations to align to 2 targets at once
    When using this function, make sure that "Target.X" and "Target.Y" are the upstream camera
    position and "Target2.X"/"Target2.Y" are the downstream position
    
    mcd_1x, mcd_1y: dictionary entries for upstream mcd x and y controllers
    mcd_2x, mcd_2y: dictionary entries for downstream mcd x and y controllers
    It is assumed that the upstream mcd "aligns to" the closer position, while the downstream mcd
    "aligns to" the further position in traditional alignment
    camera: GEECS camera object
    picos: GEECS picos object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate: we use a linear approximation of steps per pixel
    to approximate where the target should be, but multiply by opt_rate
    stage_pos_a: closer numerical value for stage position
    stage_pos_b: far numerical value for stage position
    STAGE_POS1 SHOULD BE FURTHER DOWNSTREAM THAN STAGE_POS2
    lambda_1: ratio of pixels moved of stage_pos_b/stage_pos_a for mcd_1
    lambda_2: ratio of pixels moved of stage_pos_b/stage_pos_a for mcd_2
    """
    #Delta x_a = move_1a + move_2a
    #Delta x_b = lambda_1*move_1a + lambda_2*move_2a
    #move_1a means the number of pixels to move on mcd 1
    begin_time = time.time()
    
    while True:
        t1 = time.time()
        stage.set('position', stage_pos_close)
        print(f"Time for stage to move: {time.time() - t1}")
        time.sleep(3)
        print(f"Time after sleep: {time.time() - t1}")
        centroid_close_x, centroid_close_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        t1 = time.time()
        stage.set('position', stage_pos_far)
        print(f"Time for stage to move: {time.time() - t1}")
        time.sleep(3)
        print(f"Time after sleep: {time.time() - t1}")
        centroid_far_x, centroid_far_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)

        # X matrix equation (A * \vec{pos} = \vec{b})
        # A is matrix of calibrations
        # pos = positions/values of mcds
        # b = pixel difference in target vs centroid
        # Must do this for x and y direction

        # x-dir
        A = np.array([
            [1, 1],
            [lambda_1, lambda_2],
        ])
        print("Matrix A is (X-dir):", A)
        b = np.array([
            [centroid_close_x - camera.state['Target.X']],
            [centroid_far_x - camera.state['Target2.X']],
        ])
        pixel_moves_x = np.matmul(np.linalg.inv(A), b)
        
        # y-dir
        A = np.array([
            [1, 1],
            [lambda_1, lambda_2],
        ])
        print("Matrix A is (X-dir):", A)
        b = np.array([
            [centroid_close_y - camera.state['Target.Y']],
            [centroid_far_y - camera.state['Target2.Y']],
        ])
        pixel_moves_y = np.matmul(np.linalg.inv(A), b)
        
        # move x dir
        move_pico_to_pixel(camera, obj_target, opt_rate, mcd_2x, 
                           centroid_close_x + pixel_moves_x[0][0], axis='x')
        move_axicon_to_pixel(camera, obj_target, 1, mcd_1x, 
                             centroid_close_x + pixel_moves_x[0][0] + pixel_moves_x[1][0], axis='x')
        # move y dir
        move_pico_to_pixel(camera, obj_target, opt_rate, mcd_2y, 
                           centroid_close_y + pixel_moves_y[0][0], axis='y')
        move_axicon_to_pixel(camera, obj_target, 1, mcd_1y, 
                             centroid_close_y + pixel_moves_y[0][0] + pixel_moves_y[1][0], axis='y')
        
        # Check to see if moved to correct positions
        stage.set('position', stage_pos_close)
        centroid_close_x, centroid_close_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        stage.set('position', stage_pos_far)
        centroid_far_x, centroid_far_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        
        obj1 = obj_func(camera.state['Target.X'], camera.state['Target.Y'], centroid_close_x, centroid_close_y)
        obj2 = obj_func(camera.state['Target2.X'], camera.state['Target2.Y'], centroid_far_x, centroid_far_y)
        
        if (obj1 < rough_target) and (obj2 < rough_target):
            # fine alignment only possible if both positions are positive or negative w.r.t target
            if (camera.state['Target.Y'] - centroid_close_y) * (camera.state['Target2.Y'] - centroid_far_y) > 0:
                if (camera.state['Target.X'] - centroid_close_x) * (camera.state['Target2.X'] - centroid_far_x) > 0:
                    break
    
    stage.set('position', stage_pos_close)
    centroid1_x, centroid1_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    stage.set('position', stage_pos_far)
    centroid2_x, centroid2_y = get_centroid(camera, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    print("Close delta x:", centroid1_x - camera.state['Target.X'])
    print("Close delta y:", centroid1_y - camera.state['Target.Y'])
    print("Far delta x:", centroid2_x - camera.state['Target2.X'])
    print("Far delta y:", centroid2_y - camera.state['Target2.X'])
    print(f"Alignment took {time.time() - begin_time} seconds")
    return
    
    
    

In [29]:
# Solve the 4D matrix equation Ax = b
# x = [mcd_close_x, mcd_far_x, mcd_close_y, mcd_far_y]
# b = [d_target_close_x, d_target_far_x, d_target_close_y, d_target_far_y]
# A in steps/pix
A = np.array(
    [[-1.201923e-4, 1, -1.1261261e-4, 1],
    [-8.4745762e-4, 1.6286644e-4, -8.3333333e-4, 1.5197568e-4],
    [6.5e-4, 1, 6.5e-4, 1],
    [1, 6.5e-4, 1, 6.5e-4]],
)

In [31]:
align_2_targets_at_once(mcd_dict['axicon_x'], mcd_dict['axicon_y'], mcd_dict['HM_x'], mcd_dict['HM_y'], camera, 4, 
                        100, 0.5, 570, 550, A, fit='linear', fit_thresh=10)

Time for stage to move: 0.1266624927520752
Time after sleep: 3.1279120445251465
Centroid is 990.388245, 647.228821
Centroid is 988.969421, 648.170532
Centroid is 990.611816, 641.71698
Time for stage to move: 2.727724552154541
Time after sleep: 5.729942798614502
Centroid is 1005.528259, 648.185791
Centroid is 1006.924805, 647.67865
Centroid is 1006.014221, 647.863953
Matrix A is (X-dir): [[-8.32000053e+03  1.00000000e+00 -8.88000021e+03  1.00000000e+00]
 [-1.18000001e+03  6.14000036e+03 -1.20000000e+03  6.58000017e+03]
 [ 1.53846154e+03  1.00000000e+00  1.53846154e+03  1.00000000e+00]
 [ 1.00000000e+00  1.53846154e+03  1.00000000e+00  1.53846154e+03]]
axicon x new pos: -0.278669472040978
axicon y new pos: -1.0654964055172187
HM x new pos: 5.606308741300833
HM y new pos: 6.0764124544588665
Error:
	Message: Error occurred during check values -  ypos was not change acording to the command. actual value= -0.278700
	Source:  Failed to execute command "setypos", error originated in control sy

KeyboardInterrupt: 

In [246]:
def align_2_targets_at_once_cr_y(mcd1, mcd2, camera1, camera2, rough_target, 
                               opt_rate_thresh, fit='linear', fit_thresh=10):
    """
    Uses 2 different calibrations to align to 2 targets at once
    
    mcd1: list like object of 2 labels to control motion control device 1
    mcd2: list like object of 2 labels to control motion control device 2
    mcd1hex, mcd2hex: True if they are hexapods, False if they aren't
    camera: GEECS camera object
    picos: GEECS picos object
    obj_target: stop optimizing once obj_func < obj_target
    opt_rate_thresh: threshold for significantly smaller optimization, using a sigmoid function
    stage_pos1: should correspond to target1
    stage_pos2: should correspond to target2
    STAGE_POS1 SHOULD BE FURTHER DOWNSTREAM THAN STAGE_POS2
    calib_1_length: path length from mcd1 to stage_pos1
    calib_2_length: path length from mcd2 to stage_pos1
    fit: if linear, use linear calib (steps_per_pixel)
    if poly, use polynomial fit (these should be PolynomialModel instances inside picos_dict)
    fit_thresh: threshold between fine and rough poly fits
    """
    # x1 = a1 * hex_x + b1 * pico_x
    # x2 = a2 * hex_x + b2 * pico_x
    # Solve for hex x and pico x
    while True:
        centroid1_x, centroid1_y = get_centroid(camera1, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        centroid1_x, centroid2_y = get_centroid(camera2, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        
        obj1 = (camera1.state['Target.Y'] - centroid1_y)**2
        obj2 = (camera2.state['Target.Y'] - centroid2_y)**2

        print(f"obj1: {obj1}")
        print(f"obj2: {obj2}")
        if (obj1 < rough_target) and (obj2 < rough_target):
            if (camera1.state['Target.Y'] - centroid1_y) * (camera2.state['Target.Y'] - centroid2_y) > 0:
                break

        calib_mcd1_cam1_y = mcd_dict[mcd1]['calib_cam1']
        mcd1_y = mcd_dict[mcd1]

        calib_mcd2_cam1_y = mcd_dict[mcd2]['calib_cam1']
        mcd2_y = mcd_dict[mcd2]

        calib_mcd1_cam2_y = mcd_dict[mcd1]['calib_cam2']
        calib_mcd2_cam2_y = mcd_dict[mcd2]['calib_cam2']

        # X matrix equation (A * \vec{pos} = \vec{b})
        # A is matrix of calibrations
        # pos = positions/values of mcds
        # b = pixel difference in target vs centroid
        # Must do this for x and y direction

        # y-dir
        A = 1 / np.array([
            [calib_mcd1_cam1_y, calib_mcd2_cam1_y],
            [calib_mcd1_cam2_y, calib_mcd2_cam2_y]
        ]) # take reciprocal because calib numbers in steps/pixel instead of pixels/step
        b = np.array([
            [camera1.state['Target.Y'] - centroid1_y], [camera2.state['Target.Y'] - centroid2_y]
        ])
        print("Solving equation Ax=b")
        print("A =", A)
        print("b =", b)
        pos = np.matmul(np.linalg.inv(A), b)
        print("pos = ", pos)
        larger_move = max(abs(pos[0][0]), abs(pos[1][0]))
        # Sigmoid optimization rate
        opt_rate = min(1, abs(larger_move/opt_rate_thresh))
        print("Opt rate:", opt_rate)

        # Make sure move is at least 0.001
        if pos[0][0] > 0:
            mcd1_move_y = max(pos[0][0] * opt_rate, 0.001)
        else:
            mcd1_move_y = min(pos[0][0] * opt_rate, -0.001)
        if pos[1][0] > 0:
            mcd2_move_y = max(pos[1][0] * opt_rate, 0.001)
        else:
            mcd2_move_y = min(pos[1][0] * opt_rate, -0.001)

        print(f"mcd1_move_y: {mcd1_move_y}")
        print(f"mcd2_move_y: {mcd2_move_y}")
        print(mcd1_y['device'].state[mcd1_y['alias']])
        mcd1_y_new_pos = mcd1_move_y + mcd1_y['device'].state[mcd1_y['alias']]
        mcd2_y_new_pos = mcd2_move_y + mcd2_y['device'].state[mcd2_y['alias']]

        print(f"mcd1 new position: {mcd1_y_new_pos}")
        print(f"mcd2 new position: {mcd2_y_new_pos}")

    #     Move all 4 motors at once
    #     mcd1_y['device'].set(mcd1_y['name'], mcd1_y_new_pos)
    #     mcd2_y['device'].set(mcd2_y['name'], mcd2_y_new_pos)
        conex_move_hysteresis(mcd1, mcd1_y_new_pos, CONEX_HYSTERESIS_SIZE)
        time.sleep(1)
        conex_move_hysteresis(mcd2, mcd2_y_new_pos, CONEX_HYSTERESIS_SIZE)

        time.sleep(1)
        # Check to see if moved to correct positions
        centroid1_x, centroid1_y = get_centroid(camera1, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
        centroid2_x, centroid2_y = get_centroid(camera2, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)

        obj1 = (camera1.state['Target.Y'] - centroid1_y)**2
        obj2 = (camera2.state['Target.Y'] - centroid2_y)**2

        print(f"obj1: {obj1}")
        print(f"obj2: {obj2}")
        if (obj1 < rough_target) and (obj2 < rough_target):
            # fine alignment only possible if both x and y directions 
            if (camera1.state['Target.Y'] - centroid1_y) * (camera2.state['Target.Y'] - centroid2_y) > 0:
                break
                
    print("-"*20)
    print("Moving to fine alignment")
    print("-"*20)
    align_2_targets_at_once_cr_y_close(mcd1, camera1, camera2, eps=1e-3)
    
    centroid1_x, centroid1_y = get_centroid(camera1, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)
    centroid2_x, centroid2_y = get_centroid(camera2, SHOTS_TO_AVG, TIME_BETWEEN_SHOTS)

    obj1 = (camera1.state['Target.Y'] - centroid1_y)**2
    obj2 = (camera2.state['Target.Y'] - centroid2_y)**2
    
    return obj1, obj2
    
    
    

In [247]:
opt_rate_thresh = 0.005
rough_target = 100 # 10 pixels

align_2_targets_at_once_cr_y('conex1', 'conex2', camera1, camera2, rough_target,
                            opt_rate_thresh, fit='linear', fit_thresh=10)

obj1: 541.0592805837599
obj2: 329.1354295681019
Solving equation Ax=b
A = [[-2054.56596436 -3176.16984079]
 [-1180.25106351 -1596.94606332]]
b = [[ 23.260681]
 [-18.14209 ]]
pos =  [[ 0.20264955]
 [-0.13841122]]
Opt rate: 1
mcd1_move_y: 0.20264954507278427
mcd2_move_y: -0.13841121886932284
3.227016
mcd1 new position: 3.429665545072784
mcd2 new position: 4.145605781130677
new pos: 3.429665545072784, cur_pos: 3.227016
Error:
	Message: VISA Read in CONEX-PP-example.vi->conex-pp.lvclass:Configure.vi->Device_control.vi
	Source:  Failed to execute command "setposition", error originated in control system
forward 2.0902990901455683
backwards 1.685
Error:
	Message: VISA Read in CONEX-PP-example.vi->conex-pp.lvclass:Configure.vi->Device_control.vi
	Source:  Failed to execute command "setposition", error originated in control system
new pos: 4.145605781130677, cur_pos: 4.284017
obj1: 17.121024715535732
obj2: 4.491017116803885
--------------------
Moving to fine alignment
--------------------
new

(1.345395847743914, 0.033863728441010946)